In [7]:
# Dependencies
from census import Census
# from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
import imp

# Import API key
temp_file = imp.load_source("gkey", "../Project1/gkey.py")

gkey = temp_file.gkey

del temp_file

In [8]:
grocery_df = pd.read_csv(
    "grocery_stl.csv", dtype="object", encoding="utf-8")

grocery_df.head()


,Name,Address,Lat,Lng,Zip Code
0,Schnucks Arsenal,"5055 Arsenal, St. Louis",38.6068029,-90.273927,63139
1,Schnucks Lindell,"4171 Lindell Blvd, St. Louis",38.641805,-90.2472562,63108
2,Straub's,"8282 Forsyth Blvd, Clayton",38.652284,-90.344205,63105
3,Schnucks Richmond Ctr.,"SCHNUCKS RICHMOND CTR., 6600 Clayton Rd, St. L...",38.6334297,-90.31466809999999,63117
4,Schnucks South City,"3430 S Grand Blvd, St. Louis",38.5950102,-90.24251540000002,63118


In [9]:
data_df = pd.read_csv(
    "data.csv", dtype="object", encoding="utf-8")

data_df.fillna(0)


,Zip Code,Grocery Store Count,Population,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployment Rate,White,Black,Asian,White %,Black %,Asian %,Obese %,Overweight %,Overweight or Obese %,Lat,Lng
0,63118,4.0,28810.0,29880.0,19201.0,9996.0,34.696286011801455,6.626171468240194,12177.0,15464.0,1031.0,42.26657410621312,53.67580701145435,3.578618535230823,27.80%,33.90%,61.70%,38.5950619,-90.2291565
1,63108,4.0,21345.0,39162.0,37387.0,5799.0,27.167955024595923,4.928554696650269,11550.0,7684.0,2156.0,54.111033028812365,35.999063012415085,10.100726165378308,16.90%,31.20%,48.10%,38.647459,-90.25730999999999
2,63116,3.0,44175.0,41273.0,24673.0,9304.0,21.06168647425014,4.5319750990379175,30977.0,9475.0,3756.0,70.12337294850029,21.44878324844369,8.502546689303905,25.20%,33.90%,59.00%,38.5844866,-90.266699
3,63104,3.0,20320.0,47816.0,32508.0,5150.0,25.344488188976378,4.409448818897638,10432.0,9616.0,354.0,51.338582677165356,47.32283464566929,1.7421259842519685,22.20%,35.20%,57.40%,38.6160491,-90.2291565
4,63130,2.0,29488.0,63382.0,42794.0,4433.0,15.033233857840477,4.0864080303852415,16619.0,11621.0,1561.0,56.35851871947911,39.40925122083559,5.293678784590342,12.30%,29.10%,41.40%,38.6682669,-90.32308069999999
5,63139,1.0,22072.0,49887.0,34433.0,2744.0,12.432040594418268,2.328742297934034,19222.0,2087.0,932.0,87.08771293947082,9.455418629938384,4.22254440014498,22.40%,33.60%,56.00%,38.6159025,-90.2948797
6,63133,1.0,7507.0,22632.0,12815.0,2900.0,38.630611429332625,8.778473424803517,617.0,7027.0,0.0,8.218995604102837,93.60596776342081,0.0,0.00%,50.00%,50.00%,38.6787998,-90.3042778
7,63119,1.0,33894.0,72214.0,40836.0,1869.0,5.514250309789343,2.0092051690564703,30063.0,3099.0,1022.0,88.69711453354576,9.143211187820853,3.0152829409334987,31.30%,29.20%,60.40%,38.5842641,-90.3513021
8,63117,1.0,9097.0,70680.0,51963.0,666.0,7.321094866439486,2.781136638452237,7514.0,1142.0,687.0,82.59865889853798,12.553589095306146,7.551940200065956,17.70%,33.70%,51.30%,38.6289223,-90.33013419999999
9,63113,1.0,11270.0,24283.0,16087.0,4156.0,36.87666370896184,10.221827861579415,373.0,10870.0,82.0,3.3096716947648623,96.45075421472937,0.7275953859804791,38.50%,34.10%,72.60%,38.6605914,-90.24557689999999


In [10]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

In [11]:
# Store 'Lat' and 'Lng' into  locations 
locations = grocery_df[["Lat", "Lng"]].astype(float)

In [12]:
# data_df["Obese %"] = data_df["Obese %"].strip('%').strip()
house_income = data_df["Household Income"].astype(float)
locations_zip = data_df[["Lat", "Lng"]].astype(float)


In [13]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations_zip, weights=house_income, 
#                                  dissipating=False, max_intensity=100,
                                 point_radius = 25)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
# heat_layer.dissipating = False
# heat_layer.max_intensity = 100
heat_layer.point_radius = 25
heat_layer.gradient = ["rgba(102, 255, 0, 0)",
"rgba(255, 57, 0, 1)",
"rgba(255, 113, 0, 1)",
"rgba(255, 170, 0, 1)",
"rgba(249, 198, 0, 1)",
"rgba(244, 227, 0, 1)",
"rgba(238, 255, 0, 1)",
"rgba(193, 255, 0, 1)",
"rgba(147, 255, 0, 1)",
"rgba(102, 255, 0, 1)",
"rgba(102, 255, 0, 0)"]

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# markers = gmaps.marker_layer(locations)
fig = gmaps.figure()
store_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(300, 0, 0, 0.4)',
    stroke_color='green', scale=2
)
fig.add_layer(store_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(store_layer)

fig

Figure(layout=FigureLayout(height='420px'))